#組員
朱彥如 109065711
張誠晉 110062609
蘇東誠 108061123

#Summary
##Feature Extraction 
*  Channels
*  Time
*  Weekend
*  number of Images
*  released day(Sun, Mon, ...) 
*  number of links
*  number of videos
*  content
*  Word2vec(not used)
*  number of youtube links
*  time (hours/month/date/hour)
*  number of tag a
*  distance to 5 LDAs (tile)
*  distance to 5 LDAs (content)
*  keywords to tfidf
*  contens to tfidf
*  top 10 channels
*  top 50 catagories
*  ave of content non-zero tfidfs
*  number of non-zero tfidfs
*  author names

## Extarction method
use regular expression or other parsing tools

## Tried Classifiers
* SVC
* Random Forest
* Category Bayes
* SGD(log loss, hinge loss)
* Adaboost
* Xgboost

## Conclusion
1. Random Forest performs better than other models in this case, and it has lower requirment for the training data (e.g. no normalization is ok)
2. Randome Forest can perform better with higher `n_estimator` value. However, the improvement shrinks and the traing time increases as we increase `n_estimator`. Thus, we chose a number between 1000 to 1400.

3. `np.get_dummy()` is a convenient function to transform any feature, like authors, to one-hot feature

4. we can apply one dummy(one-hot) result to another by 
`dummy2 = dummy2.reindex(columns = dummy1.columns, fill_value=0)`

 



# Import things

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
from pylab import *
import pandas as pd

import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import re
from bs4 import BeautifulSoup
""
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from scipy.sparse import csr_matrix, hstack
import pickle

# Preprocessing

In [ ]:
## YR update
def preprocessor(o_text):
    r='(>[^<]*[Mm]ashable[^<]*<)|(>[^<]*MASHABLE[^<]*<)'
    o_text = re.sub(r, '><', o_text)
    # remove content in time tag
    text = BeautifulSoup(o_text)
    time_tag = text.find("time")
    # remove a
    a_tag = text.find_all("a")
    # remove em
    em_tag = text.find_all("em")
    # <div class="credit">
    credit_tag = text.find_all("div" ,class_='credit') 
    # image-credit
    image_credit_tag = text.find_all("div" ,class_='image-credit')
    # <div class="guest-author-summary">
    geust_auth_tag = text.find_all("div" ,class_='guest-author-summary')
   
    credit_tag=set(credit_tag)
    for i in credit_tag:
        o_text = o_text.replace(str(i),'')
        
    em_tag=set(em_tag)
    for i in em_tag:
        o_text = o_text.replace(str(i),'')
        
    geust_auth_tag=set(geust_auth_tag)
    for i in geust_auth_tag:
        o_text = o_text.replace(str(i),'')

    image_credit_tag=set(image_credit_tag)
    for i in image_credit_tag:
        o_text = o_text.replace(str(i),'')
    
    a_tag=set(a_tag)
    for i in a_tag:
        o_text = o_text.replace(str(i),'')
    
    text = o_text.replace(str(time_tag),'')

    # remove HTML tags
    sp = BeautifulSoup(text)
    text=sp.find('section',class_='article-content').get_text()
    #text = BeautifulSoup(text, 'html.parser').get_text()
     
    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

In [ ]:
def df_to_LDAMatrix(input_df):
    #_new_counts = count_vect.transform(input_df['Page content_text'])
    def_dtm = LDA_count_df.transform(input_df['Page content_text'])
    tr_M=LDA.transform(def_dtm)
    #tr_M = hstack([tr_M, input_df['Weekend'].values.reshape(-1, 1), input_df['Fri'].values.reshape(-1, 1), input_df['img'].values.reshape(-1, 1), input_df['pop ch'].values.reshape(-1, 1)])
    concatenate_M=np.concatenate([tr_M, input_df['Page content_text'].values.reshape(-1, 1), input_df['Fri'].values.reshape(-1, 1), input_df['img'].values.reshape(-1, 1), input_df['pop ch'].values.reshape(-1, 1)],axis=1)
    return concatenate_M

# PorterStemmer

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,1),
                        preprocessor=preprocessor,
                        tokenizer=tokenizer_stem_nostop)

In [ ]:
tfidf = pickle.load(open("/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/vectorizer.pickle", "rb"))

In [ ]:
test_tfidf = tfidf.transform(test['Page content'])
test_tfidf = hstack([test_tfidf, test['Weekend'].values.reshape(-1, 1), test['Image'].values.reshape(-1, 1)])
pred = clf.predict_proba(test_tfidf)

In [ ]:
pred1 = [round(x[1],1) for x in pred]

In [ ]:
submission = pd.DataFrame()
submission['Id'] = submit_sample['Id']
submission['Popularity'] = pred1
print(submission)
submission.to_csv("submission.csv", index = False)

#Feature Selection 0
## ""in the title
by 東誠



In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/preprocess_df.csv')
print(df["title"].head(5))


   Id  Popularity                                  Page content_text  Weekend  \
0   0          -1   clara moskowitz for space com nasa s grand ch...        0   
1   1           1  by christina warrengoogle s new open source pa...        0   
2   2           1  by sam lairdballin 2014 nfl draft picks get to...        0   
3   3          -1  by sam lairdcameraperson fails deliver slapsti...        0   
4   4          -1  by connor finnegannfl star helps young fan pro...        0   

        img                                              title      link  \
0  0.008929  NASA's Grand Challenge: Stop Asteroids From De...  0.070513   
1  0.017857  Google's New Open Source Patent Pledge: We Won...  0.057692   
2  0.017857  Ballin': 2014 NFL Draft Picks Get to Choose Th...  0.035256   
3  0.008929        Cameraperson Fails Deliver Slapstick Laughs  0.041667   
4  0.464286  NFL Star Helps Young Fan Prove Friendship With...  0.051282   

     iframe                                       Page c

In [ ]:
import re
#print(df["title"].head(5))
quot_num = []
for title in df["title"].values:
  #print(len(re.findall("\'\w+\'", title)))
  quot_num.append(len(re.findall("\'\w+\'", title)))



# Feature Selection 1
by 誠晉

*  Time(hours)
*  Weekend
*  number of Images
*  every kind of channels
*  released day(1-31)
*  released time in one-hot('0'-'24')
*  week days in one-hot('Sun'-'Sat')
*  number of links
*  number of videos
*  LDA distances
*  released month(1-12)

In [2]:
import pandas as pd
df = pd.read_csv('df.csv') # input dataframe
print(df.columns)

Index(['Id', 'Popularity', 'Page content', 'Time', 'Weekend', 'Img', 'World',
       'Tech', 'Entertain', 'Business', 'Watercooler', 'Startups', 'Music',
       'Gadgets', 'Social-media', 'Apps', 'Marketing', 'Media', 'Gaming',
       'Mobile', 'Us', 'Pics', 'Dev', 'Small-busine', 'Film', 'How-to', 'Day',
       '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       'Sun', 'Wed', 'Fri', 'Tue', 'Mon', 'Thu', 'Sat', 'Link', 'Video',
       'LDA1', 'LDA2', 'LDA3', 'LDA4', 'LDA5', 'Monthnum'],
      dtype='object')


#### Extarction method

In [ ]:
# Time
time = [str(BeautifulSoup(x).find("time"))[33:35] for x in df['Page content']]
df['Time'] = [int(x) for x in time]
for x in range(25):
    df[str(x)] = [1 if y == x else 0 for y in df['Time']]
    
# Image
df['Image'] = [len(BeautifulSoup(x).find_all("img")) for x in df['Page content']]

# Article
Article = [str(BeautifulSoup(x).find("article"))[22:35] for x in df['Page content']]
for x in {'world','tech','entertain','business','watercooler','startups','music','gadgets','social-media','apps','marketing','gaming','mobile','us','pics','dev','small-busine','film','how-to'}:
    df[x] = [1 if y.startswith(x) else 0 for y in Article]

# Day
df['Day'] = [int(str(BeautifulSoup(x).find("time"))[21:23]) for x in df['Page content']]    
    
# Weekdays
Weekdays = [str(BeautifulSoup(x).find("time"))[16:19] for x in df['Page content']]
df['Weekend'] = [1 if (x == 'Sat' or x == 'Sun') else 0 for x in Weekdays]
for x in {"Mon","Tue","Wed","Thu","Fri","Sat","Sun"}:
    df[x] = [1 if y == x else 0 for y in Weekdays]
    
# Video
df['Video'] = [len(BeautifulSoup(x).find_all("iframe")) for x in df['Page content']]

# Link
df['Link'] = [len(BeautifulSoup(x).find_all("a")) for x in df['Page content']]

# LDA
text =tfidf.transform(df['Page content'])
lda = LatentDirichletAllocation(n_components=5)
lda.fit(text)
LDA = lda.transform(text)
df['LDA1'] = [x[0] for x in LDA]
df['LDA2'] = [x[1] for x in LDA]
df['LDA3'] = [x[2] for x in LDA]
df['LDA4'] = [x[3] for x in LDA]
df['LDA5'] = [x[4] for x in LDA]

# Month
def M(x):
    if x == 'Jan':
        return 1
    if x == 'Feb':
        return 2
    if x == 'Mar':
        return 3
    if x == 'Apr':
        return 4
    if x == 'May':
        return 5
    if x == 'Jun':
        return 6
    if x == 'Jul':
        return 7
    if x == 'Aug':
        return 8
    if x == 'Sep':
        return 9
    if x == 'Oct':
        return 10
    if x == 'Nov':
        return 11
    if x == 'Dec':
        return 12
month = [str(BeautifulSoup(x).find("time"))[24:27] for x in df['Page content']]
df['Monthnum'] = [M(x) for x in month]

# Feature Selection 2
by 彥如

*  num of images
*  number of youtube links
*  time (hours/month/date/hour)
*  Mon Tues, Wed, ...
*  is weekend? 
*  number of tag a
*  distance to 5 LDAs (tile)
*  distance to 5 LDAs (content)
*  keywords to tfidf
*  contens to tfidf
*  top 10 channels
*  top 50 catagories
*  ave of content non-zero tfidfs
*  number of non-zero tfidfs
*  author names

#### Extarction method

In [ ]:
def get_time(content):
    sp=BeautifulSoup(content)
    tmp_sp=sp.find_all('time')
    if(len(tmp_sp)==0):
        return ""
    else:
        try:
            return tmp_sp[0]['datetime']
        except KeyError:
            return ""
def time_df(input_df):
    input_df['time']=input_df['Page content'].apply(get_time)
    return input_df

In [ ]:
def is_Weekend(content):
    sp=BeautifulSoup(content)
    tmp_sp=sp.find_all('time')
    if(len(tmp_sp)==0):
        return 0
    else:
        try:
            is_Sun=tmp_sp[0]['datetime'].find('Sun')
            is_Sat=tmp_sp[0]['datetime'].find('Sat')
            if (is_Sun==-1 and is_Sat==-1):
                return 0
            else:
                return 1
        except KeyError:
            return 0

In [ ]:
def catch_title(content):
    sp=BeautifulSoup(content)
    tmp_sp=sp.find_all('h1')
    title=tmp_sp[0].get_text()
    return title
def df_cathc_title(tmp_df):
    tmp_df['title']=tmp_df['Page content'].apply(catch_title)
    return tmp_df

In [ ]:
def img_count(content):
    sp=BeautifulSoup(content)
    tmp_sp=sp.find_all('img')
    return len(tmp_sp)
def img_df(input_df):
    input_df['img']=input_df['Page content'].apply(img_count)
    return input_df

In [ ]:
def ytv_count(content):
    sp=BeautifulSoup(content)
    tmp_sp=sp.find_all('iframe')
    count=0
    for i in tmp_sp:
        try:
            ind=i['src'].find('youtube')
            if (ind>-1):
                count=count+1
        except:
            continue
    return count

def ytv_df(df_tmp):
    df_tmp['youtube vedio']=df_tmp['Page content'].apply(ytv_count)
    return df_tmp

In [ ]:
def figure_count(content):
    sp=BeautifulSoup(content)
    tmp_sp=sp.find_all('figure')
    return len(tmp_sp)

def figure_df(df_tmp):
    df_tmp['figure']=df_tmp['Page content'].apply(figure_count)
    return df_tmp

In [ ]:
def link_count(content):
    sp=BeautifulSoup(content)
    tmp_sp=sp.find_all('a')
    return len(tmp_sp)

def link_df(df_tmp):
    df_tmp['link']=df_tmp['Page content'].apply(link_count)
    return df_tmp

In [ ]:
def get_channel(content):
    sp=BeautifulSoup(content)
    try:
        channel=sp.find('article')['data-channel']
        return channel          
    except:
        return ""
def channel_df(df_tmp):
    df_tmp['channel']=df_tmp['Page content'].apply(get_channel)
    return df_tmp

In [ ]:
def get_pop_ch(content):
    sp=BeautifulSoup(content)
    try:
        channel=sp.find('article')['data-channel']
        if (channel=="socmed" or channel=="sports" or channel=="comics" or channel=="memes" or channel=="viral" or channel=="mob" or channel=="howto"):
            return 1
        else:
            return 0
    except:
        return 0

In [ ]:
def get_author(content):
    sp=BeautifulSoup(content)
    try:
        tmp_sp=sp.find('span',class_="author_name").find('a').get_text()
        return tmp_sp
    except:
        return ""

In [ ]:
def tfidf_mean(content):
    x=tf_transformer.transform(cv.transform([content]))
    x_arr=np.array(x.toarray()[0])
    return x_arr.sum()/np.count_nonzero(x_arr)
def tf_mean_df(input_df):
    input_df['tf_mean']=input_df['Page content_text'].apply(tfidf_mean)
    return input_df

In [ ]:
def tfidf_count(content):
    x=tf_transformer.transform(cv.transform([content]))
    x_arr=x.toarray()[0]
    sum_=0
    for i in x_arr:
        if(i!=0):
            sum_=sum_+1
    return sum_
def tf_count_df(input_df):
    input_df['tf_mean']=input_df['Page content_text'].apply(tfidf_count)
    return input_df

In [ ]:
def get_gatgory(row):
    p=re.compile("<a href=\"/category/([a-z0-9]+)/\">")    
    return p.findall(row)
    

In [ ]:
def one_hot_cat(row,key):
    for i in row:
        if(i==key):
            return 1
    return 0
for i in range(50):
    print(sorted_counter_object[i][0])
    key=sorted_counter_object[i][0]
    pre_df[key+'_cat']=pre_df['catgory'].apply(one_hot_cat,key=key)
    pre_df[key+'_cat']=pre_df['catgory'].apply(one_hot_cat,key=key)

world
tech
entertainment
watercooler
business
us
video
mobile
gadgets
television
photography
lifestyle
sports
film
music
twitter
travel
startups
marketing
politics
apple
facebook
media
advertising
gaming
conversations
google
apps
youtube
space
jobs
humor
home
climate
iphone
android
microsoft
comics
instagram
samsung
amazon
gallery
smartphone
art
australia
ukraine
memes
celebrities
health
kickstarter


#### LDA

In [ ]:
LDA_count_df = CountVectorizer(ngram_range=(1, 1),
                        tokenizer=tokenizer_stem_nostop, stop_words = 'english')
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA_count_df.fit(pre_df['Page content_text'])
dtm = LDA_count_df.transform(pre_df['Page content_text'])
LDA.fit(dtm)

LatentDirichletAllocation(n_components=5, random_state=42)

### Feature Display

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/data v4/preprocess_df.csv') # input dataframe
print(df.columns)
print(df.loc[1,:])

Index(['Id', 'Popularity', 'Page content_text', 'Weekend', 'img', 'title',
       'link', 'Page content', 'figure', 'Fri', 'pop ch', 'world', 'socme',
       'app', 'marketing', 'author', 'time', 'Sat', 'channel', 'youtube vedio',
       'tech', 'watercooler', 'entertainment', 'startups', 'social-media',
       'tf_mean', 'year', 'month', 'date', 'hour', 'month_en', 'catgory',
       'world_cat', 'tech_cat', 'entertainment_cat', 'watercooler_cat',
       'business_cat', 'us_cat', 'video_cat', 'mobile_cat', 'gadgets_cat',
       'television_cat', 'photography_cat', 'lifestyle_cat', 'sports_cat',
       'film_cat', 'music_cat', 'twitter_cat', 'travel_cat', 'startups_cat',
       'marketing_cat', 'politics_cat', 'apple_cat', 'facebook_cat',
       'media_cat', 'advertising_cat', 'gaming_cat', 'conversations_cat',
       'google_cat', 'apps_cat', 'youtube_cat', 'space_cat', 'jobs_cat',
       'humor_cat', 'home_cat', 'climate_cat', 'iphone_cat', 'android_cat',
       'microsoft_cat', 'comi

# Saving Vectorized Data


In [ ]:
import pickle
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
df_raw = pd.read_csv('/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/dataset_comp/train.csv')
df = pd.read_csv('/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/誠晉/df_new.csv') # input dataframe
df_test = pd.read_csv('/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/誠晉/modtest.csv')
tfidf = pickle.load(open("/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/vectorizer.pickle", "rb")) # input vetorizer
X = df['Page content']
X_test = df_test['Page content']
#print(df["iframe"].idxmax())
#print(df_raw["Page content"].values[df["iframe"].idxmax()])

#X = tfidf.transform(X)
#pickle.dump(X, open("/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/X_train_content.pkl", "wb"))
X = tfidf.transform(X_test)
pickle.dump(X, open("/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/X_test_content.pkl", "wb"))
X = hstack([X, weekend.values.reshape(-1, 1), img.values.reshape(-1, 1), links.values.reshape(-1,1), videos.values.reshape(-1,1), df["Fri"].values.reshape(-1,1)])
print(X.shape)

(11847, 100)


In [ ]:
print(df['Popularity'].values[df["iframe"].idxmax()])

1


# Model Search

#SGD Classifier

In [ ]:
def get_stream(path, size):
    for chunk in pd.read_csv(path, chunksize=size):
        yield chunk

In [ ]:
#Use SGDClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC
# loss='log' gives logistic regression
clf = SGDClassifier(loss='log', max_iter=100, tol=1e-3)
batch_size = 1000
stream = get_stream(path='/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/Weekend&Img/df.csv', size=batch_size)
classes = np.array([-1, 1])
train_auc, val_auc = [], []
# we use one batch for training and another for validation in each iteration
iters = int((25000+batch_size-1)/(batch_size*2))
for i in range(iters):
    batch = next(stream)
    X_train, y_train, weekend1, img1 = batch['Page content'], batch['Popularity'], batch['Weekend'], batch['Image']
    if X_train is None:
        break
    X_train = tfidf.transform(X_train)
    X_train = hstack([X_train, weekend1.values.reshape(-1, 1), img1.values.reshape(-1, 1)])
    
    clf.partial_fit(X_train, y_train, classes=classes)
    train_auc.append(roc_auc_score(y_train, clf.predict_proba(X_train)[:,1]))
    
    # validate
    batch = next(stream)
    X_val, y_val, weekend2, img2 = batch['Page content'], batch['Popularity'], batch['Weekend'], batch['Image']
    X_val = tfidf.transform(X_val)
    X_val = hstack([X_val, weekend2.values.reshape(-1, 1), img2.values.reshape(-1, 1)])
    score = roc_auc_score(y_val, clf.predict_proba(X_val)[:,1])
    val_auc.append(score)
    print('[{}/{}] {}'.format((i+1)*(batch_size*2), 25000, score))

[2000/25000] 0.48958783686410146
[4000/25000] 0.5382308965771573
[6000/25000] 0.516750067000268


KeyboardInterrupt: ignored

# SVC

* C=0.1, gamma=1, kernel=poly;, score=0.55625
* C=0.1, gamma=0.1, kernel=poly; score=0.555
* C=0.1, gamma=0.01, kernel=poly; score=0.555
* C=1, gamma=1, kernel=poly; score=0.535
* C=1, gamma=0.1, kernel=poly; score=0.5525
* C=1, gamma=0.01, kernel=poly; score=0.555
* C=10, gamma=1, kernel=poly, score = 0.512

* C=0.1, gamma=

In [ ]:
from sklearn.model_selection import GridSearchCV
import pickle
import pandas as pd
from sklearn.svm import SVC, LinearSVC


X = pickle.load(open("/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/X_train_content.pkl", "rb"))
df = pd.read_csv('/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/誠晉/df_new.csv')
y = df['Popularity']

param_grid = {'C': [0.1,1, 10], 'gamma': [1,0.1,0.01],'kernel': ['linear', 'rbf']}

grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=4, scoring='roc_auc', cv=4)
grid.fit(X,y)
print(grid.best_estimator_)


Fitting 4 folds for each of 18 candidates, totalling 72 fits
[CV 1/4] END .....C=0.1, gamma=1, kernel=linear;, score=0.557 total time= 3.7min
[CV 2/4] END .....C=0.1, gamma=1, kernel=linear;, score=0.549 total time= 3.8min
[CV 3/4] END .....C=0.1, gamma=1, kernel=linear;, score=0.541 total time= 3.7min
[CV 4/4] END .....C=0.1, gamma=1, kernel=linear;, score=0.557 total time= 3.8min
[CV 1/4] END ........C=0.1, gamma=1, kernel=rbf;, score=0.557 total time= 4.2min
[CV 2/4] END ........C=0.1, gamma=1, kernel=rbf;, score=0.556 total time= 4.2min
[CV 3/4] END ........C=0.1, gamma=1, kernel=rbf;, score=0.548 total time= 4.2min
[CV 4/4] END ........C=0.1, gamma=1, kernel=rbf;, score=0.563 total time= 4.2min
[CV 1/4] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.557 total time= 3.8min
[CV 2/4] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.549 total time= 3.8min
[CV 3/4] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.541 total time= 3.8min
[CV 4/4] END ...C=0.1, gamma=0.1, kernel=linear;

#Adaboost

In [ ]:
from sklearn.model_selection import GridSearchCV
import pickle
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

X = pickle.load(open("/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/X_train_content.pkl", "rb"))
df = pd.read_csv('/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/Weekend&Img/df.csv')
y = df['Popularity']
abc = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())

parameters = {'base_estimator__max_depth':[i for i in range(2,11,2)],
              'base_estimator__min_samples_leaf':[5,10],
              'n_estimators':[10,50,250,1000],
              'learning_rate':[0.01,0.1]}

grid = GridSearchCV(abc, parameters ,verbose=4, scoring='roc_auc' ,n_jobs=-1, cv=4)
grid.fit(X,y)
print(grid.best_estimator_)

Fitting 4 folds for each of 80 candidates, totalling 320 fits


KeyboardInterrupt: ignored

#**Decision Trees** (Final Choice)
##content, img, , weekend
best: min_samples_leaf=6, n_estimators=20, random_state=11, criterion=entropy, max_dept=5
##content, img, , weekend, link
best: max_depth=4, n_estimators=30, random_state=11, criterion=entropy
##content, img, weekend, link, videos, Fri
{'criterion': 'gini', 'max_depth': 6, 'n_estimators': 800} auc = 0.5647
##content, img, weekend, Fri
{'criterion': 'gini', 'max_depth': 6, 'n_estimators': 800
## all features expect LDAs
{'criterion': 'gini', 'max_depth': 9, 'n_estimators': 1000} auc = 0.56775
## all features
{'criterion': 'gini', 'max_depth': 12, 'n_estimators': 1500}

{'criterion': 'gini', 'max_depth': 11, 'n_estimators': 1500} auc = 0.568

## all features + month + year + date
{'criterion': 'gini', 'max_depth': 11, 'n_estimators': 1000} auc = 0.5795
## all features + month + year + date + catagories + author + title
{'criterion': 'gini', 'max_depth': 11, 'n_estimators': 1400 auc = 0.584

In [ ]:
from sklearn.model_selection import GridSearchCV
import pickle
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pdb
#quot_num = np.array(quot_num)
df = pd.read_csv('/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/誠晉/df_new.csv') # input dataframe

df_2 = pd.read_csv('/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/data v4/preprocess_df.csv')

X = pickle.load(open("/content/drive/MyDrive/大四上資料/Deep_learning /COMP1/X_train_content.pkl", "rb"))
y, weekend, img, links, videos, sat = df['Popularity'], df['Weekend'], df['Img'], df["Link"], df["Fri"], df['Sat']
title = tfidf.transform(df_2.loc[:, "title"]).toarray()
XX = np.hstack([X.toarray(), df.loc[:,'World':'How-to'].values, df.loc[:,'Weekend':'Img'].values, \
               df.loc[:,"0":"Video"].values, df.loc[:,"LDA1":"LDA5"].values,\
               pd.get_dummies(df_2['month_en']).values, pd.get_dummies(df_2['year']).values,\
               pd.get_dummies(df_2['date']).values, pd.get_dummies(df_2['hour']).values, \
               title, pd.get_dummies(df_2["author"]), df_2.loc[:,"world_cat": "kickstarter_cat"].values])
X_noauthor = np.hstack([X.toarray(), df.loc[:,'World':'How-to'].values, df.loc[:,'Weekend':'Img'].values, \
               df.loc[:,"0":"Video"].values, df.loc[:,"LDA1":"LDA5"].values,\
               pd.get_dummies(df_2['month_en']).values, pd.get_dummies(df_2['year']).values,\
               pd.get_dummies(df_2['date']).values, pd.get_dummies(df_2['hour']).values, \
               title, df_2.loc[:,"world_cat": "kickstarter_cat"].values])
#X2 = np.hstack([X.toarray(), df.iloc[:, ]]) 
print(X_noauthor.shape)


y = df['Popularity']

param_grid = {    
    "n_estimators":[10,15,20],
    "criterion": ["gini", "entropy"],
    "min_samples_leaf": [2,4,6],
    "max_depth": [2,3,4,'None']}
param_grid2 = {
    "n_estimators":[100], 
    "criterion": ["entropy", "gini"],
    "max_depth": [3,4,5,6]
}
param_grid3 = {
    "n_estimators":[1000, 1400], 
    "criterion": ["gini"],
    "max_depth": [10, 11]
}
clf = RandomForestClassifier(random_state = 11)
grid = GridSearchCV(clf, param_grid3,refit=True,verbose=5, scoring='roc_auc', cv=4)
grid.fit(X_noauthor,y)
print(grid.best_params_)

(27643, 381)
Fitting 4 folds for each of 4 candidates, totalling 16 fits
[CV 1/4] END criterion=gini, max_depth=10, n_estimators=1000;, score=0.581 total time= 1.3min
[CV 2/4] END criterion=gini, max_depth=10, n_estimators=1000;, score=0.568 total time= 1.3min


KeyboardInterrupt: ignored

[CV 1/4] END criterion=gini, max_depth=6, n_estimators=800;, score=0.566 total time=  31.9s

[CV 2/4] END criterion=gini, max_depth=6, n_estimators=800;, score=0.559 total time=  35.6s

[CV 3/4] END criterion=gini, max_depth=6, n_estimators=800;, score=0.564 total time=  31.5s

[CV 4/4] END criterion=gini, max_depth=6, n_estimators=800;, score=0.568 total time=  31.2s


[CV 1/4] END criterion=gini, max_depth=8, n_estimators=1000;, score=0.567 total time=  50.2s

[CV 2/4] END criterion=gini, max_depth=8, n_estimators=1000;, score=0.560 total time=  50.4s

[CV 3/4] END criterion=gini, max_depth=8, n_estimators=1000;, score=0.567 total time=  50.4s

[CV 4/4] END criterion=gini, max_depth=8, n_estimators=1000;, score=0.571 total time=  50.8s


[CV 1/4] END criterion=gini, max_depth=10, n_estimators=1000;, score=0.567 total time= 1.1min

[CV 2/4] END criterion=gini, max_depth=10, n_estimators=1000;, score=0.560 total time= 1.2min

[CV 3/4] END criterion=gini, max_depth=10, n_estimators=1000;, score=0.569 total time= 1.1min

[CV 4/4] END criterion=gini, max_depth=10, n_estimators=1000;, score=0.576 total time= 1.2min


In [ ]:
print(grid.best_params_)

{'criterion': 'gini', 'max_depth': 6, 'n_estimators': 30}


## Our Final Choice

Best result by 誠晉   
Used 'Page content', 'Time', 'Weekend', 'Img', 'World',  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'Tech', 'Entertain', 'Business', 'Watercooler', 'Startups', 'Music',  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'Gadgets', 'Social-media', 'Apps', 'Marketing', 'Media', 'Gaming',  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'Mobile', 'Us', 'Pics', 'Dev', 'Small-busine', 'Film', 'How-to', 'Day',  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'Sun', 'Wed', 'Fri', 'Tue', 'Mon', 'Thu', 'Sat', 'Link', 'Video',  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'LDA1', 'LDA2', 'LDA3', 'LDA4', 'LDA5', 'Monthnum' as features.  
Page content been tfidf transformed, and all other features duplicated twice.  
Used random forrest as model, with n_estimator = 2000 and max_depth = 14

In [3]:
import pandas as pd
df = pd.read_csv('df.csv') # input dataframe
print(df.columns)

Index(['Id', 'Popularity', 'Page content', 'Time', 'Weekend', 'Img', 'World',
       'Tech', 'Entertain', 'Business', 'Watercooler', 'Startups', 'Music',
       'Gadgets', 'Social-media', 'Apps', 'Marketing', 'Media', 'Gaming',
       'Mobile', 'Us', 'Pics', 'Dev', 'Small-busine', 'Film', 'How-to', 'Day',
       '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       'Sun', 'Wed', 'Fri', 'Tue', 'Mon', 'Thu', 'Sat', 'Link', 'Video',
       'LDA1', 'LDA2', 'LDA3', 'LDA4', 'LDA5', 'Monthnum'],
      dtype='object')


In [4]:
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
from pylab import *
import pandas as pd

import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import re
from bs4 import BeautifulSoup
""
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from scipy.sparse import csr_matrix, hstack
import pickle
# my functions
import helpers.data_mining_helpers as dmh
from sklearn.preprocessing import StandardScaler # scikit-learn 0.21.3
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
def preprocessor(o_text):
    # remove content in time tag
    text = BeautifulSoup(o_text)
    time_tag = text.find("time")
    text = o_text.replace(str(time_tag),'')

    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
     
    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

tfidf1 = TfidfVectorizer(max_features = 1000,
                         ngram_range=(1,1),
                         preprocessor=preprocessor,
                         tokenizer=tokenizer_stem_nostop)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ccc87\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
tfidf1.fit(df['Page content'])

TfidfVectorizer(max_features=1000,
                preprocessor=<function preprocessor at 0x000001A4018C1D30>,
                tokenizer=<function tokenizer_stem_nostop at 0x000001A42DC9CCA0>)

In [6]:
X_train = tfidf1.transform(df['Page content'])
X_train1 = hstack([X_train, df.loc[:,'Time':'Video'].values, df.loc[:,"LDA1":"LDA5"].values, df['Monthnum'].values.reshape(-1, 1)])
X_train2 = hstack([X_train1, df.loc[:,'Time':'Video'].values, df.loc[:,"LDA1":"LDA5"].values, df['Monthnum'].values.reshape(-1, 1)])

In [ ]:
#Use RandomForest
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier(criterion='gini',max_depth=14,n_estimators=2000,n_jobs=-1)
scores = cross_validate(estimator=clf, X=X_train2, y=df['Popularity'], cv=4, scoring='roc_auc',return_train_score=True)
print('cross_tra_score: %.3f (+/-%.3f),  cross_val_score: %.3f (+/-%.3f)' % (scores['train_score'].mean(), scores['train_score'].std(), scores['test_score'].mean(), scores['test_score'].std()))

## Prediction & Save

In [ ]:
test = pd.read_csv('modtest.csv')
X_test = test['Page content']
X_test = tfidf1.transform(X_test)
X_test1 = hstack([X_test, test.loc[:,'Time':'Video'].values, test.loc[:,"LDA1":"LDA5"].values, test['Monthnum'].values.reshape(-1, 1)])
X_test2 = hstack([X_test1, test.loc[:,'Time':'Video'].values, test.loc[:,"LDA1":"LDA5"].values, test['Monthnum'].values.reshape(-1, 1)])
pred = clf.predict_proba(X_test2)
pred1 = [x[1] for x in pred]
submission = pd.DataFrame()
submission['Id'] = test['Id']
submission['Popularity'] = pred1
print(submission)
submission.to_csv("submission.csv", index = False)

### Repeat features 

The reason we hstack twice time in both training and testing data is because we find a fun fact that adding all features double times would benefit the training.
